In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

def generate_summary(df: pd.DataFrame, name: str, sample_size: int = 3) -> None:
    """Генерация расширенной сводки по данным с примерами"""
    print(f"\n{'='*50} {name.upper()} {'='*50}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")
    
    # Типы данных
    print("\nТипы данных:")
    print(df.dtypes.value_counts().rename('count').to_frame())
    
    # Пропуски
    missing = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing / df.shape[0] * 100).round(2)
    missing_df = pd.concat([missing, missing_pct], axis=1, keys=['count', '%']).query('count > 0')
    if not missing_df.empty:
        print("\nПропущенные значения:")
        print(missing_df)
    else:
        print("\nПропущенных значений нет")
        
    # Дубликаты
    dupes = df.duplicated().sum()
    print(f"\nДубликаты: {dupes} ({dupes/df.shape[0]*100:.2f}%)")
    
    # Примеры
    print(f"\nПервые {sample_size} записей:")
    display(df.head(sample_size))


def load_data(data_path: Path) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Загрузка данных из PKL-файлов"""
    sessions = pd.read_pickle(data_path / 'ga_sessions.pkl')
    hits     = pd.read_pickle(data_path / 'ga_hits.pkl')
    return sessions, hits


def optimize_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """Оптимизация типов данных"""
    # Объектные → category (если уникальных существенно меньше числа строк)
    for col in df.select_dtypes(include='object'):
        if df[col].nunique() < df.shape[0] * 0.5:
            df[col] = df[col].astype('category')
    # Пытаться downcast чисел
    for col in df.select_dtypes(include=['int64', 'float64']):
        try:
            df[col] = pd.to_numeric(df[col], downcast='integer')
        except Exception:
            pass
    return df


def process_datetime_sessions(df: pd.DataFrame, date_col: str, time_col: str) -> pd.DataFrame:
    """Обработка даты и времени визита в единый session_start"""
    df = df.copy()
    if date_col in df:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    if time_col in df:
        df[time_col] = pd.to_datetime(
            df[time_col], format='%H:%M:%S', errors='coerce'
        ).dt.time
    df['session_start'] = pd.to_datetime(
        df[date_col].dt.strftime('%Y-%m-%d') + ' ' + df[time_col].astype(str),
        errors='coerce'
    )
    # Оставляем только годы 2020–2023
    df = df[df['session_start'].dt.year.between(2020, 2023)]
    return df


def process_datetime_hits(df: pd.DataFrame) -> pd.DataFrame:
    """
    Обработка событий: 
    — hit_date → date; 
    — hit_time_ms = смещение (ms) от начала сессии.
    """
    df = df.copy()
    if 'hit_date' in df:
        df['hit_date'] = pd.to_datetime(df['hit_date'], errors='coerce').dt.date
    # Конвертация в целочисленные миллисекунды
    df['hit_time_ms'] = pd.to_numeric(df['hit_time'], errors='coerce').fillna(0).astype(int)
    return df


def handle_device_data(df: pd.DataFrame) -> pd.DataFrame:
    """Очистка и разбор device_screen_resolution → screen_width, screen_height"""
    df = df.fillna({
        'device_os': 'unknown',
        'device_brand': 'unknown',
        'device_model': 'generic',
        'device_browser': 'unknown',
        'device_screen_resolution': '0x0'
    })
    mask = df['device_screen_resolution'].str.match(r'^\d+x\d+$', na=False)
    df.loc[~mask, 'device_screen_resolution'] = '0x0'
    df[['screen_width', 'screen_height']] = (
        df['device_screen_resolution']
          .str.split('x', expand=True)
          .astype(int)
    )
    df = df.drop(columns=['device_screen_resolution'])
    return df


def process_utm(df: pd.DataFrame) -> pd.DataFrame:
    """Очистка UTM-меток и преобразование в категории"""
    utm_map = {
        'utm_source':   {'(none)': 'organic', '(not set)': 'organic', 'nan': 'organic'},
        'utm_medium':   {'(none)': 'organic', 'not set': 'organic', 'nan': 'organic'},
        'utm_campaign': {'(not set)': 'general', 'nan': 'general'},
        'utm_adcontent':{'(not set)': 'none', 'nan': 'none'},
        'utm_keyword':  {'(not set)': 'none', 'nan': 'none'}
    }
    for col, mapping in utm_map.items():
        if col in df:
            df[col] = df[col].replace(mapping).fillna('other').astype('category')
    return df


def add_conversion_target(sessions: pd.DataFrame, hits: pd.DataFrame) -> pd.DataFrame:
    """Добавление флага конверсии per session_id"""
    actions = ['sub_button_click','sub_page_view','sub_view_cars_click','sub_landing']
    hits = hits.copy()
    hits['is_conversion'] = hits['event_action'].isin(actions).astype(int)
    conv = hits.groupby('session_id', as_index=False)['is_conversion'].max()
    return sessions.merge(conv, on='session_id', how='left').fillna({'is_conversion': 0})


def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """Удаление дубликатов и полностью пустых колонок"""
    df = df.drop_duplicates()
    df = df.dropna(axis=1, how='all')
    return df[df['session_id'].notna()]


def main_pipeline(data_path: Path, save_path: Path) -> None:
    # 1. Загрузка
    sessions, hits = load_data(data_path)
    
    # 2. Обработка сессий
    sessions_proc = (
        sessions
        .pipe(clean_data)
        .pipe(process_datetime_sessions, 'visit_date', 'visit_time')
        .pipe(handle_device_data)
        .pipe(process_utm)
        .pipe(optimize_dtypes)
    )
    
    # 3. Обработка событий
    hits_proc = (
        hits
        .pipe(clean_data)
        .pipe(process_datetime_hits)
        .pipe(optimize_dtypes)
    )
    
    # 4. Вычисляем истинное время каждого хита
    merged = hits_proc.merge(
        sessions_proc[['session_id','session_start']],
        on='session_id', how='left'
    )
    merged['hit_datetime'] = merged['session_start'] + pd.to_timedelta(merged['hit_time_ms'], unit='ms')
    hits_proc = merged.drop(columns=['session_start'])
    
    # 5. Добавляем целевую переменную в сессии
    sessions_proc = add_conversion_target(sessions_proc, hits_proc)
    
    # 6. Сохраняем результаты
    save_path.mkdir(parents=True, exist_ok=True)
    sessions_proc.to_pickle(save_path / 'processed_sessions.pkl')
    hits_proc.to_pickle(save_path / 'processed_hits.pkl')


if __name__ == "__main__":
    root     = Path(os.getcwd()).parent.parent
    data_raw = root / 'data' / 'raw_data'
    data_out = root / 'data' / 'processed_data'
    main_pipeline(data_raw, data_out)
